In [1]:
!pip install transformers
!pip install torch-summary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.5 MB/s eta 0:00:00


In [2]:
import torch
from torch import nn
from transformers import BertModel , BertTokenizer, DistilBertTokenizer , DistilBertModel
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim



In [3]:
device = torch.device("cuda:0")

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def parse_input(input_seq):
  x = tokenizer(input_seq,padding='max_length', max_length = 512,
                       truncation=True, return_tensors="pt")
  return {'input_ids':x['input_ids'].to(device),'attention_mask':x['attention_mask'].to(device)}


def one_hot(size,index):
  return torch.eye(size)[index-1].cuda()


In [ ]:
print(one_hot(8,1).device)
print(parse_input("my name is rem"))

cuda:0
{'input_ids': tensor([[ 101, 1139, 1271, 1110, 1231, 1306,  102,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
       

## Dataset

In [5]:
import json

# Opening JSON file
with open('/content/drive/MyDrive/Colab Notebooks/Text-Classifier/data.json') as json_file:
    data = json.load(json_file)

class myDataset(Dataset):

  def __init__(self,data):
    self.labels = [one_hot(8,obj.get('label')) for obj in data.get('train')]
    self.texts = [parse_input(obj.get('input')) for obj in data.get('train')]

  def classes(self):
    return self.labels

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, index):
    return self.texts[index], self.labels[index]


train_dataset = myDataset(data)

train_loader = DataLoader(train_dataset,batch_size=1,shuffle=True)

In [ ]:
_,label = next(iter(train_loader))
label.dtype

torch.float32

## The model

In [6]:
class TextClassifier(nn.Module):

    def __init__(self, dropout=0.2):

        super(TextClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')
        # self.dropout = nn.Dropout(dropout)
        self.linear_1 = nn.Linear(768, 8)
        self.relu_1 = nn.ReLU()

    def forward(self, seq):
        _, pooled_output = self.bert(input_ids= seq.get('input_ids').squeeze(1), attention_mask=seq.get('attention_mask'),return_dict=False)
        linear_output = self.linear_1(pooled_output)
        final_layer = self.relu_1(linear_output)
        return final_layer

model = TextClassifier()
model.to(device)
print(next(model.parameters()).device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


cuda:0


## training loop

In [10]:
num_epochs=200
criterion = nn.CrossEntropyLoss()
criterion = criterion.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.00002,betas=(0.9,0.999),weight_decay=0.1)

In [ ]:
for epoch in range(num_epochs):
  for batch_idx, (seq, label) in enumerate(train_loader):
    optimizer.zero_grad() # zero gradients
    output = model(seq)
    loss = criterion(output.squeeze(), label.squeeze()) # calculate loss
    loss.backward() # backward pass
    optimizer.step()
  # if (epoch+1) % 5 == 0:
  print('Epoch: {}, Batch: {}, Loss: {:.4f}'.format(epoch+1, batch_idx, loss.item()))

Epoch: 1, Batch: 88, Loss: 2.1810
Epoch: 2, Batch: 88, Loss: 2.2149
Epoch: 3, Batch: 88, Loss: 1.5061
Epoch: 4, Batch: 88, Loss: 1.3704
Epoch: 5, Batch: 88, Loss: 0.4487
Epoch: 6, Batch: 88, Loss: 1.8825
Epoch: 7, Batch: 88, Loss: 0.7147
Epoch: 8, Batch: 88, Loss: 0.1502
Epoch: 9, Batch: 88, Loss: 0.2480
Epoch: 10, Batch: 88, Loss: 0.1444
Epoch: 11, Batch: 88, Loss: 0.0943
Epoch: 12, Batch: 88, Loss: 0.0543
Epoch: 13, Batch: 88, Loss: 0.2714
Epoch: 14, Batch: 88, Loss: 0.0995
Epoch: 15, Batch: 88, Loss: 0.1182
Epoch: 16, Batch: 88, Loss: 0.2864
Epoch: 17, Batch: 88, Loss: 0.1288
Epoch: 18, Batch: 88, Loss: 0.0695
Epoch: 19, Batch: 88, Loss: 0.0470
Epoch: 20, Batch: 88, Loss: 0.0712
Epoch: 21, Batch: 88, Loss: 0.1626
Epoch: 22, Batch: 88, Loss: 0.0951
Epoch: 23, Batch: 88, Loss: 0.0622
Epoch: 24, Batch: 88, Loss: 0.0683
Epoch: 25, Batch: 88, Loss: 0.0625
Epoch: 26, Batch: 88, Loss: 0.0769
Epoch: 27, Batch: 88, Loss: 0.0777
Epoch: 28, Batch: 88, Loss: 0.0624
Epoch: 29, Batch: 88, Loss: 0

In [ ]:
torch.save(model.state_dict(), "IntentionRecognition_model_cpu.pt")

In [ ]:
model = model.to('cuda')

In [10]:
summary(model,input_size=(1,512))

Layer (type:depth-idx)                   Param #
├─BertModel: 1-1                         --
|    └─BertEmbeddings: 2-1               --
|    |    └─Embedding: 3-1               22,268,928
|    |    └─Embedding: 3-2               393,216
|    |    └─Embedding: 3-3               1,536
|    |    └─LayerNorm: 3-4               1,536
|    |    └─Dropout: 3-5                 --
|    └─BertEncoder: 2-2                  --
|    |    └─ModuleList: 3-6              85,054,464
|    └─BertPooler: 2-3                   --
|    |    └─Linear: 3-7                  590,592
|    |    └─Tanh: 3-8                    --
├─Dropout: 1-2                           --
├─Linear: 1-3                            6,152
├─ReLU: 1-4                              --
Total params: 108,316,424
Trainable params: 108,316,424
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─BertModel: 1-1                         --
|    └─BertEmbeddings: 2-1               --
|    |    └─Embedding: 3-1               22,268,928
|    |    └─Embedding: 3-2               393,216
|    |    └─Embedding: 3-3               1,536
|    |    └─LayerNorm: 3-4               1,536
|    |    └─Dropout: 3-5                 --
|    └─BertEncoder: 2-2                  --
|    |    └─ModuleList: 3-6              85,054,464
|    └─BertPooler: 2-3                   --
|    |    └─Linear: 3-7                  590,592
|    |    └─Tanh: 3-8                    --
├─Dropout: 1-2                           --
├─Linear: 1-3                            6,152
├─ReLU: 1-4                              --
Total params: 108,316,424
Trainable params: 108,316,424
Non-trainable params: 0

In [ ]:
example = "Rem, Illuminate the front of the car"
example = parse_input(example)
mask = example['attention_mask'].to(device)
input_ids = example['input_ids'].squeeze(1).to(device)
example={'input_ids':input_ids,'attention_mask':mask}
prediction = model(example)
max_index = torch.argmax(prediction)
print(prediction)
print(f'class:{max_index+1}')

tensor([[0.0000, 0.0000, 0.0985, 0.0000, 5.5473, 0.0114, 0.2684, 0.3219]],
       device='cuda:0', grad_fn=<ReluBackward0>)
class:5


In [ ]:
# model = model.cuda()
type(model)

__main__.TextClassifier

In [15]:
# my_trained_model = model.load('/content/drive/MyDrive/Colab Notebooks/Text-Classifier/IntentionRecognition_model_cpu.pt')
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/Text-Classifier/IntentionRecognition_model_cpu.pt', map_location='cuda'))

<All keys matched successfully>

In [22]:
example = "deactivate the door"
example = parse_input(example)
mask = example['attention_mask'].to(device)
input_ids = example['input_ids'].squeeze(1).to(device)
example={'input_ids':input_ids,'attention_mask':mask}
prediction = model(example)
print(prediction)


tensor([[2.3865, 3.7145, 0.0000, 1.9656, 0.0000, 1.6615, 0.0000, 0.0000]],
       device='cuda:0', grad_fn=<ReluBackward0>)
